In [4]:
import pandas as pd


def read_iob2_file(path):
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

# creating the dictionary of labels and their indices
class Vocab():
    def __init__(self, pad_unk='<PAD>'):
        self.pad_unk = pad_unk
        self.word2idx = {}
        self.idx2word = []

    def getIdx(self, word, add=False):
        if word is None or word == self.pad_unk:
            return None
        if word not in self.word2idx:
            if add:
                idx = len(self.idx2word)
                self.word2idx[word] = idx
                self.idx2word.append(word)
                return idx
            else:
                return None
        return self.word2idx[word]

    def getWord(self, idx):
        return self.idx2word[idx]

In [15]:
train_data = ["data\sk_snk-ud-dev.iob2", 
              "data\en_ewt-ud-train.iob2",
              ]


for train in train_data:
    train_data = read_iob2_file(train)
    lang = train[5:7]
  
    # get the tag dictionary
    label_indices = Vocab()
    tags_column = train_data["tags"]

    for tags in tags_column:
        for tag in tags:
            label_indices.getIdx(tag, add=True)

    label_list = sorted(label_indices.idx2word)
    print("Labels for", lang, ": ", label_list)

Labels for sk :  ['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']
Labels for en :  ['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']
